In [9]:
# Functional imports
import os 
import numpy as np
import nibabel as nib

from data_prep_helpers import make_vol
import fdr_correction_helpers

# Notebook specific imports
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")
%autosave 5

Autosaving every 5 seconds


In [3]:
# Set data location
directory = os.path.join(os.path.expanduser('~'), 'Documents', 'Data')
print('Data location:', directory)

Data location: /Users/angira/Documents/Data


In [4]:
# Load scores
social_only_adults = np.load(os.path.join(
    directory, 'adults', 'encoding_results', 'unthresholded', 'adults_social_only_scores_gm_mask.npy'))
print('Adult scores shape:', np.shape(social_only_adults))

social_only_34yrs = np.load(os.path.join(
    directory, '3-4-yrs', 'encoding_results', 'unthresholded', '3-4-yrs_social_only_scores_gm_mask.npy'))
print('3-4 year old scores shape:', np.shape(social_only_34yrs))

Adult scores shape: (33, 367709)
3-4 year old scores shape: (31, 367709)


In [10]:
group1 = social_only_adults
group2 = social_only_34yrs
null_dist = []
for i in range(2):
    flipped_group1= []
    for subject in range(len(group1)):
        flipped_group1.append(fdr_correction_helpers.rand_sign_flip(group1[subject]))
        print('Appended subject', subject)

    flipped_group2= []
    for subject in range(len(group2)):
        flipped_group2.append(fdr_correction_helpers.rand_sign_flip(group2[subject]))
        print('Appended subject', subject)

    # Calculate difference in means
    null_dist.append(np.subtract(np.mean(flipped_group1, axis=0), np.mean(flipped_group2, axis=0)))
    print('Completed permutation')
null_dist = np.swapaxes(null_dist, 0, 1)
print(np.shape(null_dist))

Appended subject 0
Appended subject 1
Appended subject 2
Appended subject 3
Appended subject 4
Appended subject 5
Appended subject 6
Appended subject 7
Appended subject 8
Appended subject 9
Appended subject 10
Appended subject 11
Appended subject 12
Appended subject 13
Appended subject 14
Appended subject 15
Appended subject 16
Appended subject 17
Appended subject 18
Appended subject 19
Appended subject 20
Appended subject 21
Appended subject 22
Appended subject 23
Appended subject 24
Appended subject 25
Appended subject 26
Appended subject 27
Appended subject 28
Appended subject 29
Appended subject 30
Appended subject 31
Appended subject 32
Appended subject 0
Appended subject 1
Appended subject 2
Appended subject 3
Appended subject 4
Appended subject 5
Appended subject 6
Appended subject 7
Appended subject 8
Appended subject 9
Appended subject 10
Appended subject 11
Appended subject 12
Appended subject 13
Appended subject 14
Appended subject 15
Appended subject 16
Appended subject 17


In [12]:
print(np.shape(null_dist))
np.save(os.path.join(directory, 'null_dists', 'null_dist1.npy'), null_dist)
np.save(os.path.join(directory, 'null_dists', 'null_dist2.npy'), null_dist)
np.save(os.path.join(directory, 'null_dists', 'null_dist3.npy'), null_dist)
print('Saved fake null dists successfully')

(367709, 2)
Saved fake null dists successfully


In [23]:
file_directory = os.path.join(directory, 'null_dists')
all_files = os.listdir(file_directory)

distributions = []

# Ensure only .npy files are stored (remove .DS_Store, etc)
for filename in all_files:  
    if filename[-4:] in ('.npy'):
        dist_name = os.path.join(file_directory, filename)
        dist = np.load(dist_name)
        distributions.append(dist)
    else:
        print('discarding file: ', filename)

# Ensure formatting such that iteration is over dists, not voxels
distributions = np.swapaxes(distributions, 1, 2)

# Perform concatenation
concatenated_dist = []
for i in range(len(distributions)):
    for dist in distributions[i]:
        concatenated_dist.append(dist) 

return np.swapaxes(concatenated_dist, 0, 1)
print(np.shape(concatenated_dist))

discarding file:  .DS_Store
(367709, 6)


(367709, 6)
